In [1]:
from pennylane import numpy as np
import pandas as pd
import pickle
from utils import (GenerateTermsMatrix, GenerateConstrainTerms, solve_qubo)


np.random.seed(412)

In [16]:

# PARAMETERS 
STRUCTURE = "WarrenTruss_5"  # or "WarrenTruss_11"
N_S = 2 if "5" in STRUCTURE else 4
ALPHA = 1

# Read data
with open(f'data/{STRUCTURE}.pickle', 'rb') as handle:
        data = pickle.load(handle)

# extract matrices
stiffness_matrix = data["K"]
mass_matrix = data["M"]
modal_matrix = data["Phi"]
N = len(stiffness_matrix)

# Modal participation factors
activeDof = data["FreeDOFList"]
r = np.asarray([1 for i in range(len(activeDof))])
Meff = (modal_matrix.T @ mass_matrix @ r)**2
MMP = Meff / np.sum(Meff)
MMP_index = list(range(len(MMP)))
MMP_sorted, MMP_index_sorted = zip(*sorted(zip(MMP, MMP_index), reverse=True))
print(f"MMP_index_sorted: {MMP_index_sorted}")
print(f"MMP_sorted: {MMP_sorted}")
index = np.argmax(np.cumsum(MMP_sorted) > 0.95)
print(index)
MODES = MMP_index_sorted[:index+1]
print(MODES)
print(f"Total MMP: {sum([MMP[i] for i in MODES])}")



MMP_index_sorted: (0, 1, 4, 3, 2, 5)
MMP_sorted: (tensor(0.47393058, requires_grad=True), tensor(0.44243766, requires_grad=True), tensor(0.05001517, requires_grad=True), tensor(0.02539565, requires_grad=True), tensor(0.00754717, requires_grad=True), tensor(0.00067377, requires_grad=True))
2
(0, 1, 4)
Total MMP: 0.9663834116273986


In [17]:
stiffness_matrix = data["K"]
mass_matrix = data["M"]
modal_matrix = data["Phi"][:, MODES] # Modes to reach 95% MMP
N = len(stiffness_matrix)

# Generate the MSE terms
A = GenerateTermsMatrix(modal_matrix, stiffness_matrix)
normalization_constant = (np.ones((N,1)).T @ A @ np.ones((N,1)))[0][0] # Heuristic: normalize by max energy (all dof with sensors), which is always easy to compute.
print(f"Normalization constant: {normalization_constant}")
A /= normalization_constant

# Generate constraint terms
constrain_terms, constrain_offset = GenerateConstrainTerms(N, N_S) 

# solve the QUBO
df = solve_qubo(A - ALPHA * constrain_terms)
df["f_obj + constraint offset"] = df.f_obj - ALPHA * constrain_offset
display(df.head(50))
display(df.tail(50))

Normalization constant: 89869457.23703745


,Candidate Solutions,Candidate Solutions Index,f_obj,f_obj + constraint offset
0,"(1, 0, 0, 0, 1, 0)","[0, 4]",4.289216834384899,0.28921683438489865
1,"(0, 1, 1, 0, 0, 0)","[1, 2]",4.288494256108369,0.2884942561083692
2,"(0, 0, 1, 0, 0, 1)","[2, 5]",4.288494256108368,0.2884942561083683
3,"(0, 0, 1, 0, 1, 0)","[2, 4]",4.262043428187491,0.26204342818749105
4,"(1, 0, 1, 0, 0, 0)","[0, 2]",4.262043428187491,0.26204342818749105
5,"(0, 1, 0, 1, 0, 0)","[1, 3]",4.1875423367172715,0.1875423367172715
6,"(0, 0, 0, 1, 0, 1)","[3, 5]",4.187542336717271,0.1875423367172706
7,"(0, 0, 1, 1, 0, 0)","[2, 3]",4.1842169704178644,0.18421697041786445
8,"(0, 1, 0, 0, 0, 1)","[1, 5]",4.1815529684713955,0.18155296847139546
9,"(0, 0, 0, 0, 1, 1)","[4, 5]",4.1775415345511675,0.1775415345511675


,Candidate Solutions,Candidate Solutions Index,f_obj,f_obj + constraint offset
14,"(1, 0, 0, 1, 0, 0)","[0, 3]",4.157943602287855,0.15794360228785465
15,"(1, 0, 1, 0, 1, 0)","[0, 2, 4]",3.4935754349624513,-0.5064245650375487
16,"(0, 1, 1, 0, 0, 1)","[1, 2, 5]",3.430790357050248,-0.569209642949752
17,"(1, 1, 1, 0, 0, 0)","[0, 1, 2]",3.40433952912937,-0.5956604708706301
18,"(0, 1, 1, 0, 1, 0)","[1, 2, 4]",3.40433952912937,-0.5956604708706301
19,"(1, 0, 1, 0, 0, 1)","[0, 2, 5]",3.4043395291293694,-0.5956604708706306
20,"(0, 0, 1, 0, 1, 1)","[2, 4, 5]",3.4043395291293694,-0.5956604708706306
21,"(1, 0, 0, 1, 1, 0)","[0, 3, 4]",3.3935551230782934,-0.6064448769217066
22,"(0, 1, 1, 1, 0, 0)","[1, 2, 3]",3.3852601085899425,-0.6147398914100575
23,"(0, 0, 1, 1, 0, 1)","[2, 3, 5]",3.3852601085899416,-0.6147398914100584
